In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../../')

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import datetime

from data.processing.processing import get_data

from models.seir import SEIRHD, SEIRHD_t

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.uncertainty import MCUncertainty
from main.seir.optimiser import Optimiser
from main.seir.forecast import get_forecast, create_region_csv, create_all_csvs, write_csv, forecast_all_trials, trials_to_df, scale_up_testing_and_forecast
from main.seir.sensitivity import gridsearch_single_param, calculate_sensitivity_and_plot

from utils.generic.create_report import save_dict_and_create_report
from utils.loss import Loss_Calculator
from utils.enums import Columns
from utils.enums.columns import *

from viz import axis_formatter, plot_forecast

In [ ]:
df_state = get_data(state='Delhi', district=None, disable_tracker=False, use_dataframe='data_all', filename=None)

In [ ]:
df_state

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
for compartment in compartments['base']:
    ax.plot(df_state[compartments['date'][0].name].to_numpy(), 
            df_state[compartment.name].to_numpy(),
            '-o', color=compartment.color, label='{} (Observed)'.format(compartment.label))
axis_formatter(ax)
plt.tight_layout()

In [ ]:
df_state['daily_cases'] = df_state['total'].diff()
df_state['daily_tests'] = df_state['tested'].diff()
df_state.dropna(axis=0, how='any', subset=['daily_tests'], inplace=True)
df_state = df_state[df_state['daily_tests'] > 0]

In [ ]:
fig, axs = plt.subplots(figsize=(18, 12), nrows=2)
axs[0].plot(df_state[compartments['date'][0].name].to_numpy(), 
        df_state['daily_tests'].to_numpy(),
        '--o', color='purple', label='{} (Observed)'.format('Daily Tests'))
axs[0].plot(df_state[compartments['date'][0].name].to_numpy(), 
        df_state['daily_tests'].rolling(7, center=True).mean().to_numpy(),
        '-', color='purple', label='{} (RA)'.format('Daily Tests'))
axs[1].plot(df_state[compartments['date'][0].name].to_numpy(), 
        df_state['daily_cases'].to_numpy(),
        '--o', color='indigo', label='{} (Observed)'.format('Daily New Cases'))
axs[1].plot(df_state[compartments['date'][0].name].to_numpy(), 
        df_state['daily_cases'].rolling(7, center=True).mean().to_numpy(),
        '-', color='indigo', label='{} (RA)'.format('Daily New Cases'))

axis_formatter(axs[0])
axis_formatter(axs[1])
plt.tight_layout()

In [ ]:
predictions_dict = {}

In [ ]:
state, district = ('Maharashtra', 'Mumbai')

In [ ]:
predictions_dict['m1'] = single_fitting_cycle(
    'Delhi', None, data_from_tracker=True, granular_data=False, filename=None, #Data
    model=SEIRHD, #Choose Model and Ranges
    train_period=21, val_period=50, num_evals=1000, #Optimisation related parameters
    which_compartments=['total', 'active', 'recovered', 'deceased'], #Compartments to Apply Loss on 
    smooth_jump=True)

predictions_dict['state'] = state
predictions_dict['dist'] = district
predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
predictions_dict['datasource'] = 'covid19api' if predictions_dict['m1']['data_from_tracker'] else 'municipality'
predictions_dict['variable_param_ranges'] = predictions_dict['m1']['variable_param_ranges']

In [ ]:
predictions_dict['m1']['best_params']

In [ ]:
extra_args = {
    'time_varying_start_date': '2020-06-06',
    'factor_to_increase_gamma': 2,
    'time_varying_length': 10
}

In [ ]:
best_params = copy.copy(predictions_dict['m1']['best_params'])
best_params['T_recov_severe'] =  22
best_params['I_hosp_ratio'] =  0.8

In [ ]:
params_dict = {**best_params, **predictions_dict['m1']['default_params'], **extra_args}

In [ ]:
solver = SEIRHD_t(**params_dict)
df_prediction = solver.predict(total_days=71)

In [ ]:
df_prediction

In [ ]:
df_true_plotting_rolling = pd.concat([predictions_dict['m1']['df_train'], predictions_dict['m1']['df_val']], ignore_index=True)
df_true_plotting = copy.copy(predictions_dict['m1']['df_district'])
df_train = copy.copy(predictions_dict['m1']['df_train'])
df_val = copy.copy(predictions_dict['m1']['df_val'])
train_period = predictions_dict['m1']['run_params']['train_period']

fig, ax = plt.subplots(figsize=(12, 12))
fig.suptitle('{}, {}'.format(district, state))
for compartment in compartments['base']:
    ax.plot(df_true_plotting[compartments['date'][0].name].to_numpy(), 
            df_true_plotting[compartment.name].to_numpy(),
            '-o', color=compartment.color, label='{} (Observed)'.format(compartment.label))
    ax.plot(df_true_plotting_rolling[compartments['date'][0].name].to_numpy(), 
            df_true_plotting_rolling[compartment.name].to_numpy(),
            '-', color=compartment.color, label='{} (Obs RA)'.format(compartment.label))
    ax.plot(df_prediction[compartments['date'][0].name].to_numpy(), 
            df_prediction[compartment.name].to_numpy(),
            '-.', color=compartment.color, label='{} (Predicted)'.format(compartment.label))

ax.axvline(x=df_train.iloc[-train_period, :]['date'], ls=':', color='brown', label='Train starts')
ax.axvline(x=df_val.iloc[0, ]['date'], ls=':', color='black', label='Val starts')

axis_formatter(ax)

In [ ]:
df_true_plotting_rolling['daily_cases'] = df_true_plotting_rolling['total'].diff()
df_true_plotting['daily_cases'] = df_true_plotting['total'].diff()
df_prediction['daily_cases'] = df_prediction['total'].diff()

fig, ax = plt.subplots(figsize=(12, 12))
fig.suptitle('{}, {}'.format(district, state))
for compartment in compartments['base_diff']:
    ax.plot(df_true_plotting[compartments['date'][0].name].to_numpy(), 
            df_true_plotting[compartment.name].to_numpy(),
            '-o', color=compartment.color, label='{} (Observed)'.format(compartment.label))
    ax.plot(df_true_plotting_rolling[compartments['date'][0].name].to_numpy(), 
            df_true_plotting_rolling[compartment.name].to_numpy(),
            '-', color=compartment.color, label='{} (Obs RA)'.format(compartment.label))
    ax.plot(df_prediction[compartments['date'][0].name].to_numpy(), 
            df_prediction[compartment.name].to_numpy(),
            '-.', color=compartment.color, label='{} (Predicted)'.format(compartment.label))

ax.axvline(x=df_train.iloc[-train_period, :]['date'], ls=':', color='brown', label='Train starts')
ax.axvline(x=df_val.iloc[0, ]['date'], ls=':', color='black', label='Val starts')

axis_formatter(ax)